# สร้างระบบจำแนกประเภทข้อความภาษาไทยด้วย scikit-learn กับ PyThaiNLP

บทเรียนนี้จะช่วย guide วิธีการสร้าง**ระบบจำแนกประเภทข้อความ**หรือ **text classification** สำหรับภาษาไทยด้วย scikit-learn กับ PyThaiNLP ซึ่งเป็น machine learning เบื้องต้น ไม่จำเป็นต้องอาศัยคอมพิวเตอรืที่มี GPU ในการทำงาน


ชุดข้อมูลที่จะใช้งานในบทเรียนนี้ คือ ชุดข้อมูล Wongnai Review เป็นชุดข้อมูลข้อความภาษาไทยรีวิวร้านอาหารพร้อมกับคะแนนดาวที่รีวิว

สิ่งที่เราจะทำ คือ ระบบจำแนกจำนวนดาวจากข้อความรีวิวภาษาไทย

อ่านเพิ่มเติม
- ชุดข้อมูล Wongnai Review: https://huggingface.co/datasets/wongnai_reviews

In [1]:
!pip install -q scikit-learn pythainlp datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00


สำหรับการโหลดชุดข้อมูล บทเรียนนี้จะให้คุณโหลดชุดข้อมูลผ่าน Hugging Face Datasets ซึ่งปัจจุบันถือเป็น HUB ในการเก็บข้อมูลสำหรับงานปัญญาประดิษฐ์ จึงข้ามขั้นตอนการอ่านไฟล์ไป

In [1]:
from datasets import load_dataset  # สำหรับโหลดชุดข้อมูล
from pythainlp.tokenize import word_tokenize  # สำหรับตัดตำภาษาไทย

In [2]:
dataset = load_dataset("wongnai_reviews")  # โหลดชุดข้อมูล Wongnai Review

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ตัวแปร dataset ที่เก็บข้อมูลที่ Wongnai Review ที่ดึงจาก Hugging Face Dataset จะมี col review_body ข้อความรีวิว (str) กับ star_rating เป็นจำนวนดาวของข้อความ (int)

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review_body', 'star_rating'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['review_body', 'star_rating'],
        num_rows: 6203
    })
})

In [4]:
# ดึงข้อมูลทั้งจาก train test มาเก็บสำหรับคู่ x,y
train_x = dataset["train"]["review_body"]
train_y = dataset["train"]["star_rating"]
test_x = dataset["test"]["review_body"]
test_y = dataset["test"]["star_rating"]

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

In [6]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
    ('vect', CountVectorizer(tokenizer=word_tokenize)),  # แปลงข้อความในอยู่ในรูป feature vectors ใช้ตัวตัดของ pytahinlp
    ('tfidf', TfidfTransformer()), # Transform a count matrix to a normalized tf or tf-idf representation
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)), # linear support vector machine (SVM)
])

In [7]:
text_clf.fit(train_x, train_y)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function word_tokenize at 0x78ecc07ea050>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [8]:
predicted = text_clf.predict(test_x) # วัดผลบน test

In [9]:
print(metrics.classification_report(test_y,predicted))

              precision    recall  f1-score   support

           0       0.67      0.04      0.08        50
           1       0.62      0.04      0.07       206
           2       0.60      0.22      0.32      1906
           3       0.52      0.95      0.67      2977
           4       0.47      0.01      0.02      1064

    accuracy                           0.53      6203
   macro avg       0.58      0.25      0.23      6203
weighted avg       0.54      0.53      0.42      6203



จะเห็นได้ว่าโมเดลนี้วัดผลบน test ได้ F1-score 53% คุณสามารถปรับปรุงโดยเลือกโมเดล ใน scikit-learn ที่ดีกว่าในบทเรียนนี้ หรือ จะไปใช้พวก deep learning

อ่านเพิ่มเติม:
- Working With Text Data https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
- Hugging Face Datasets https://huggingface.co/docs/datasets/index